In [ ]:
import os
import cv2
import numpy as np
from PIL import Image, ImageEnhance, ImageFilter, ImageOps
import matplotlib.pyplot as plt
import pytesseract
import re
import pandas as pd

# Function to display image using matplotlib
def show_image(img, title="Image"):
    plt.figure(figsize=(10, 6))
    plt.imshow(img, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

def process_image_turn(image_path, file):
    img = Image.open(image_path)
    img = img.convert('RGB')  # Ensure the image is in RGB format
    img = np.array(img)  # Convert the PIL image to a NumPy array

    ##################### rotate
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Apply edge detection
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)

    # Find lines using HoughLines
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 200)

    if lines is None:
        return pd.DataFrame(columns=['ID', 'OCR', 'target'])

    # Calculate the angle of the skew
    angles = []
    for line in lines:
        rho, theta = line[0]
        angle = np.degrees(theta) - 90
        angles.append(angle)

    # Compute the median angle
    median_angle = np.median(angles)

    # Rotate the image to correct the skew
    (h, w) = img.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, median_angle, 1.0)
    rotated = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

    all_results = []

    # Convert back to PIL image to display
    rotated_pil = Image.fromarray(cv2.cvtColor(rotated, cv2.COLOR_BGR2RGB))

    def process_and_append(image, description):
        df = process_image_with_ocr(image, file)
        if not df.empty:
            print('*****************',file)
            print(description, df['OCR'], df['target'])
            all_results.append(df)
            return True
        return False

    if process_and_append(rotated_pil, "rotated_pil"):
        return all_results
    
    rotated_270 = rotated_pil.rotate(270, expand=True)
    if process_and_append(rotated_270, "rotated_270"):
        return all_results
    
    rotated_180 = rotated_pil.rotate(180, expand=True)
    if process_and_append(rotated_180, "rotated_180"):
        return all_results
    
    rotated_90 = rotated_pil.rotate(90, expand=True)
    if process_and_append(rotated_90, "rotated_90"):
        return all_results
    
    

    mirrored_pil1 = ImageOps.mirror(rotated_pil)
    if process_and_append(mirrored_pil1, "mirrored_pil1"):
        return all_results
    
    mirrored_rotated_270 = mirrored_pil1.rotate(270, expand=True)
    if process_and_append(mirrored_rotated_270, "mirrored_rotated_270"):
        return all_results
    
    mirrored_rotated_180 = mirrored_pil1.rotate(180, expand=True)
    if process_and_append(mirrored_rotated_180, "mirrored_rotated_180"):
        return all_results
    
    mirrored_rotated_90 = mirrored_pil1.rotate(90, expand=True)
    if process_and_append(mirrored_rotated_90, "mirrored_rotated_90"):
        return all_results


    return all_results

# Function to process an image with OCR
def process_image_with_ocr(img, image_name):
    if img is None:
        return pd.DataFrame(columns=['ID', 'OCR', 'target'])

    img_cropped_cv = np.array(img)
    img_cropped_cv = cv2.cvtColor(img_cropped_cv, cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(img_cropped_cv, cv2.COLOR_BGR2GRAY)
    gray = cv2.fastNlMeansDenoising(gray, h=3)
    _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    kernel = np.ones((5, 5), np.uint8)
    dilated = cv2.dilate(binary, kernel, iterations=1)
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    x_min = x
    y_min = y
    x_max = x + w
    y_max = y + h
    padding = 50
    y_min = max(0, y_min - padding)
    y_max = min(gray.shape[0], y_max + padding)
    cropped_img = gray[y_min:y_max, x_min:x_max]
    enhancer = ImageEnhance.Contrast(Image.fromarray(cropped_img))
    enhanced_img = enhancer.enhance(2)
    enhanced_img = enhanced_img.filter(ImageFilter.SHARPEN)
    enhanced_img = enhanced_img.resize((enhanced_img.width * 2, enhanced_img.height * 2), Image.BICUBIC)
    show_image(enhanced_img)

    custom_config = r'--oem 3 --psm 6'
    text = pytesseract.image_to_string(enhanced_img, lang='kor', config=custom_config)
    text = re.sub(r'\s+', ' ', text).strip()
    text = text.replace(" ", "")
    
    text2 = pytesseract.image_to_string(gray, lang='kor', config=custom_config)
    text2 = re.sub(r'\s+', ' ', text2).strip()
    text2 = text2.replace(" ", "")

    keywords_3 = ['퇴원','원확인서']
    keywords_4 = ['진단서']
    keywords_5= ['운전','면허증']
    keywords_7 = ['통원']
    keywords_8= ['주민등록증']
    keywords_9= ['대한민국','여권']
    keywords_10 = ['납입']
    keywords_11= ['영수증','복약','조제약']
    keywords_13= ['이력서','자소서']
    keywords_14 = ['소견서']
    keywords_15= ['자동차등록증']
    keywords_12= ['처방전']
    keywords_1= ['건강보험','임신','출산']
    keywords_6 = ['외래', '계산서']
    
    keywords_dict = {
        
        3: keywords_3,
        4: keywords_4,
        5: keywords_5,
        
        7: keywords_7,
        8: keywords_8,
        9: keywords_9,
        10: keywords_10,
        11: keywords_11,
        
        13: keywords_13,
        14: keywords_14,
        15: keywords_15,
        12: keywords_12,
        1: keywords_1,
        6: keywords_6,
    }

    results = []
    for number, keywords in keywords_dict.items():
        if any(keyword in text for keyword in keywords):
            results.append((image_name, text, number))
            break
        elif any(keyword in text2 for keyword in keywords):
            results.append((image_name, text2, number))
            break

    df = pd.DataFrame(results, columns=['ID', 'OCR', 'target'])
    return df

pd.set_option('display.max_colwidth', None)

# Function to process all images in a directory
def process_directory(directory):
    all_results = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(root, file)
                results = process_image_turn(image_path, file)
                if len(results) > 0:
                    all_results.extend(results)
                              
    if all_results:
        final_df = pd.concat(all_results, ignore_index=True)
    else:
        final_df = pd.DataFrame(columns=['ID', 'OCR', 'target'])
    return final_df

# Directory containing the images
directory = '/data/ephemeral/home/datasets_fin/ocr_test3'

# Process all images in the directory
result_df = process_directory(directory)

# Save the result to a CSV file
# result_df.to_csv(r'/data/ephemeral/home/notebook/ocr_test3.csv', index=False)

# Display the final result DataFrame
print("Final Result DataFrame:")
print(result_df)
